In [ ]:
# https://www.kaggle.com/caesarlupum/deduping-record-linkage
# https://bergvca.github.io/2017/10/14/super-fast-string-matching.html?source=post_page-----84f2bfd0c536----------------------

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import time
from ftfy import fix_text
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np

C:\Users\tdornigg\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
## Data where company-names are unclean
## Goal: Match this names with a list of clean company-names
data = pd.read_excel('alt_fertig.xlsx')

In [14]:
data.head()

,Firma,Betrag
0,"""Neue Heimat"" Gemeinnützige Heizergasse 1 ...",964.82
1,"""Unser Lagerhaus"" Warenhandels GmbH Suedring...",3370.52
2,3 L Gastronomie GmbH Hoheneggerstrasse 36 ...,1191.66
3,A & R Mode-Handels GmbH Amraser-See-Str. 56a...,592.64
4,A & R Mode-Handels GmbH Amraser-See-Strasse ...,296.66


In [19]:
## Load in clean company-names
clean_org_names = pd.read_excel('companies.xlsx')

In [20]:
clean_org_names

,Firma
0,ABC - Arbeit und Bildung
1,A & R Mode-Handels GmbH
2,Unser Lagerhaus GmbH
3,Neue Heimat
4,A.Zaussinger Bau und Obervisnitz
5,A. Ö. KH der Barmherzigen Brüder
6,A. Ö. Krankenhaus des DT. Ordens
7,3 L Gastronomie GmbH
8,A.Ö. Krankenhaus der Elisabethinen


In [15]:
# Compares, if the first string is similar to the second string
# A ratio of  --> 92% (very much similar)
fuzz.ratio('A & R Mode-Handels GmbH   Amraser-See-Str. 56a', 'A & R Mode-Handels GmbH   Amraser-See-Strasse')

92

In [16]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [17]:
print('All 3-grams in "McDonalds":')
ngrams('McDonalds')

All 3-grams in "McDonalds":


[' Mc', 'Mcd', 'cdo', 'don', 'ona', 'nal', 'ald', 'lds', 'ds ']

In [18]:
# The code to generate the matrix of TF-IDF values for each is shown below.
company_names = data['Firma'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [22]:
org_name_clean = clean_org_names['Firma'].unique()

print('Vecorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(org_name_clean)
print('Vecorizing completed...')

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

org_column = 'Firma' #column to match against in the messy data
unique_org = set(data[org_column].values)

Vecorizing the data - this could take a few minutes for large datasets...
Vecorizing completed...


In [23]:
###matching query:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)

unique_org = list(unique_org) #need to convert back to a list
print('finding matches...')
matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), clean_org_names.values[j][0][0],unique_org[i]]
    matches.append(temp)

print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name'])
print('Done') 

getting nearest n...
COMPLETED IN: 0.1335599422454834
finding matches...
Building data frame...
Done


In [24]:
matches.head(10)

,Match confidence (lower is better),Matched name,Origional name
0,0.71,Unser Lagerhaus GmbH,"""Unser Lagerhaus"" Warenhandels GmbH Suedring..."
1,0.38,A. Ö. KH der Barmherzigen Brüder,A. Ö. KH DER BARMHERZIGEN BRÜDER SPITALGASSE...
2,0.33,A.Zaussinger Bau und Obervisnitz,A.Zaussinger Bau- und Obervisnitz 8 A-4224...
3,0.39,3 L Gastronomie GmbH,3 L Gastronomie GmbH Hoheneggerstrasse 36 ...
4,0.32,A & R Mode-Handels GmbH,A & R Mode-Handels GmbH Amraser-See-Str. 56a...
5,0.32,A & R Mode-Handels GmbH,A & R Mode-Handels GmbH Amraser-See-Strasse ...
6,0.69,ABC - Arbeit und Bildung,ABC - Arbeit u. Bildung Neunkirchner Strasse...
7,0.49,A. Ö. Krankenhaus des DT. Ordens,A. Ö. KRANKENHAUS D. DT. ORDENS ST. VEITER S...
8,0.55,Neue Heimat,"""Neue Heimat"" Gemeinnützige Heizergasse 1 ..."
9,0.32,A.Ö. Krankenhaus der Elisabethinen,A.Ö. KRANKENHAUS DER ELISABETHINEN VÖLKERMAR...
